In [ ]:
'''
what a decorator does is replacement, thus 
when a function is decorated by a class, 
its type becomes an instance of a class
and by access its instance attribute we are 
accessing the attribute of the decorator class
'''
class Mycache:
    name = 'race'
    def __init__(self, func):
        self.func = func
        self.cache = {}
    
    def __call__(self, *args):
        if not args in self.cache:
            self.cache[args] = self.func(*args)
        return self.cache[args]
@Mycache
def fib(n):
    return fib(n-1) + fib(n-2) if n>1 else 1
print(type(fib))
print(fib.name)

In [25]:
import inspect
from typing import Union,Optional,Dict,Any,Callable
from functools import wraps, update_wrapper, partial
from core.config import settings
from core.manager import manager

'''
put @Publish(subscriber=subscriber) above a class would make it a publisher class 
which enables it to monitor what it sent to the msg-center class and send messages to subscribers

put @Publish.register(event = event) above a method inside the class would sign the function up 
with the event
'''


class Publisher:
    def __init__(self, subscriber = manager, events = settings.EVENTS):
        # if cls and  not inspect.isclass(cls):
        #     raise TypeError('Publisher needs to be a class')
        self.subscriber = subscriber
        self.events = events

    def __call__(self, cls=None):
        self.cls = cls
        self.channel = {event: dict() for event in self.events}
        for event in self.channel:
            self.channel[event]['subscriber'] = self.subscriber

        update_wrapper(self, cls)
        def wrapper(*args, **kwargs):
            value = cls(*args, **kwargs)
            return value
        return wrapper

    @classmethod
    def register(self, event=None, subscriber=None):
        self.channel[event]['subscriber'] = subscriber or self.subscriber
        
    # class register(Publisher):
    #     def __init__(self, func=None,*,event:str=None):
    #         print(super().get_channel())
    #         self.func = func
    #         update_wrapper(self, func)

    #     def __call__(self, *args, **kwargs):
    #         return self.func(*args,**kwargs)

    #     def __get__(self, instance, owner):
    #         return partial(self.__call__, instance)


@Publisher(subscriber=manager)
class Sales:
    def __init__(self, events=None, **kwargs: Dict[str, Any]):
        self.sto_data = pd.DataFrame()

    def sales_sql(self, country: str):
        _sales_sql = f'''country:{country}
                      '''
        return _sales_sql

sales = Sales()

In [58]:
class Decorator(object):
    def __init__(self, arg):
        self.arg = arg
    def __call__(self, cls):
        class Wrapped(cls):
            classattr = self.arg

            def new_method(self, value):
                return value * 2
        return Wrapped

@Decorator("decorated class")
class TestClass(object):
    name:str = 'race'
    def __init__(self, at):
        self.at = at
    def new_method(self, value):
        return value * 3

In [68]:
import pika
from core.config import settings
cp = settings.RABBITMQ_CONNECTION_PARAMETER
con = pika.BlockingConnection(cp)
channel = con.channel()



class Broker:
    def __init__(self):
        self.msg_queue = []
        self.subscribers = {}

#self.msg_queue = {'topic':[]}
#self.subscribers = {'topic':'subscriber'}

    def notify(self, msg):
        self.msg_queue.append(msg)

    def subscribe(self, msg, subscriber):
        self.subscribers.setdefault(msg, []).append(subscriber)

    def unsubscribe(self, msg, subscriber):
        self.subscribers[msg].remove(subscriber)

    def update(self):
        for msg in self.msg_queue:
            for sub in self.subscribers.get(msg, []):
                sub.run(msg)
        self.msg_queue = []


class Publisher:
    def __init__(self, broker):
        self.broker = broker

    def publish(self, msg):
        self.broker.notify(msg)


class Subscriber:
    def __init__(self, name, broker):
        self.name = name
        self.broker = broker

    def subscribe(self, msg):
        self.broker.subscribe(msg, self)

    def unsubscribe(self, msg):
        self.broker.unsubscribe(msg, self)

    def run(self, msg):
        print(f"{self.name} got {msg}")


In [1]:
from core.broker import broker, Publisher, Subscriber

In [4]:
pub = Publisher(event='main')
pub.publish(method='sql_update',event='main',body='update_finished')

In [7]:
pub.publish(method='sql_update',event='main',body='123')

In [2]:
hex(id(broker.channel))

'0x231c4a45910'

In [3]:
hex(id(broker))

'0x231b7fd4d30'

In [9]:
from core.publisher import Publisher

In [10]:
pub = Publisher()

In [14]:
pub.publish(body='200')

sent message: "200" 
                to queue: main 
                with topic: <BasicProperties(['content_type=sql_update'])>
